## Tokenization doesn't have to be slow !

### Introduction

Before going deep into any Machine Learning or Deep Learning Natural Language Processing models, every practitioner
should find a way to map raw input strings to a representation understandable by a trainable model.

One very simple approach would be to split inputs over every space and assign an identifier to each word. This approach
would look similar to the code below in python

```python
s = "very long corpus..."
words = s.split(" ")  # Split over space
vocabulary = dict(enumerate(set(words)))  # Map storing the word to it's corresponding id
```

This approach might work well if your vocabulary remains small as it would store every word (or **token**) present in your original
input. Moreover, word variations like "cat" and "cats" would not share the same identifiers even if their meaning is 
quite close.

![tokenization_simple](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/11/tokenization.png)

### Subtoken Tokenization

To overcome the issues described above, recent works have been done on tokenization, leveraging "subtoken" tokenization.
**Subtokens** extends the previous splitting strategy to furthermore explode a word into grammatically logicial sub-components learned
from the data.

Taking our previous example of the words __cat__ and __cats__, a sub-tokenization of the word __cats__ would be [cat, ##s]. Where the prefix _"##"_ indicates a subtoken of the initial input. 
Such training algorithms might extract sub-tokens such as _"##ing"_, _"##ed"_ over English corpus.

As you might think of, this kind of sub-tokens construction leveraging compositions of _"pieces"_ overall reduces the size
of the vocabulary you have to carry to train a Machine Learning model. On the other side, as one token might be exploded
into multiple subtokens, the input of your model might increase and become an issue on model with non-linear complexity over the input sequence's length. 
 
![subtokenization](https://nlp.fast.ai/images/multifit_vocabularies.png)
 
Among all the tokenization algorithms, we can highlight a few subtokens algorithms used in Transformers-based SoTA models : 

- [Byte Pair Encoding (BPE) - Neural Machine Translation of Rare Words with Subword Units (Sennrich et al., 2015)](https://arxiv.org/abs/1508.07909)
- [Word Piece - Japanese and Korean voice search (Schuster, M., and Nakajima, K., 2015)](https://research.google/pubs/pub37842/)
- [Unigram Language Model - Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates (Kudo, T., 2018)](https://arxiv.org/abs/1804.10959)
- [Sentence Piece - A simple and language independent subword tokenizer and detokenizer for Neural Text Processing (Taku Kudo and John Richardson, 2018)](https://arxiv.org/abs/1808.06226)

Going through all of them is out of the scope of this notebook, so we will just highlight how you can use them.

### @huggingface/tokenizers library 
Along with the transformers library, we @huggingface provide a blazing fast tokenization library
able to train, tokenize and decode dozens of Gb/s of text on a common multi-core machine.

The library is written in Rust allowing us to take full advantage of multi-core parallel computations in a native and memory-aware way, on-top of which 
we provide bindings for Python and NodeJS (more bindings may be added in the future). 

We designed the library so that it provides all the required blocks to create end-to-end tokenizers in an interchangeable way. In that sense, we provide
these various components: 

- **Normalizer**: Executes all the initial transformations over the initial input string. For example when you need to
lowercase some text, maybe strip it, or even apply one of the common unicode normalization process, you will add a Normalizer. 
- **PreTokenizer**: In charge of splitting the initial input string. That's the component that decides where and how to
pre-segment the origin string. The simplest example would be like we saw before, to simply split on spaces.
- **Model**: Handles all the sub-token discovery and generation, this part is trainable and really dependant
 of your input data.
- **Post-Processor**: Provides advanced construction features to be compatible with some of the Transformers-based SoTA
models. For instance, for BERT it would wrap the tokenized sentence around [CLS] and [SEP] tokens.
- **Decoder**: In charge of mapping back a tokenized input to the original string. The decoder is usually chosen according
to the `PreTokenizer` we used previously.
- **Trainer**: Provides training capabilities to each model.

For each of the components above we provide multiple implementations:

- **Normalizer**: Lowercase, Unicode (NFD, NFKD, NFC, NFKC), Bert, Strip, ...
- **PreTokenizer**: ByteLevel, WhitespaceSplit, CharDelimiterSplit, Metaspace, ...
- **Model**: WordLevel, BPE, WordPiece
- **Post-Processor**: BertProcessor, ...
- **Decoder**: WordLevel, BPE, WordPiece, ...

All of these building blocks can be combined to create working tokenization pipelines. 
In the next section we will go over our first pipeline.

Alright, now we are ready to implement our first tokenization pipeline through `tokenizers`. 

For this, we will train a Byte-Pair Encoding (BPE) tokenizer on a quite small input for the purpose of this notebook. You can train your tokenizer from a [Dataset](https://huggingface.co/docs/datasets/package_reference/main_classes.html#dataset) instance or from text files. We will see together the first method.

For this example, we propose to use "wikitext-2-raw-v1" from [Wikitext dataset](https://huggingface.co/datasets/wikitext) which contains only 4.50 MB of text, but you can of course use a much larger dataset to train your tokenizer! 

In [ ]:
!pip install tokenizers datasets

In [2]:
from datasets import load_dataset

# Initialize a dataset
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

# Build an iterator over this dataset
def batch_iterator():
    batch_length = 1000
    for i in range(0, len(dataset), batch_length):
        yield dataset[i : i + batch_length]["text"]

Reusing dataset wikitext (/home/lucile/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)


Now that we have our training data we need to create the overall pipeline for the tokenizer. For this we have two options: 
- Solution 1: use one of the high-level classes encapsulating the overall pipeline for various well-known tokenization algorithm. The available tokenizers are listed [here](https://github.com/huggingface/tokenizers/tree/master/bindings/python/py_src/tokenizers/implementations).
- Solution 2: if you want to do something more custom, you can define the tokenizer yourself.

For both cases, the pipeline will be the same. It will be necessary first to instantiate a tokenizer architecture before training it.

We will see here how to create a Byte-level BPE tokenizer as introduced by OpenAI with their GPT-2 model with the 2 methods and train them on the dataset loaded above.

In [28]:
# Set training parameter

# the size of our tokens vocabulary
vocab_size = 25000

# the list of special tokens to have into our vocabulary. Several models use special tokens to mark the beginning of a sentence, the end of a sentence or to pad a sequence of tokens to a certain length. These special tokens must also belong to the vocabulary.
special_tokens = ["<|endoftext|>"]

#### Solution 1: Use a tokenization pipeline already implemented

In [29]:
from tokenizers import ByteLevelBPETokenizer

# Initialize a not train tokenizer
tokenizer_1 = ByteLevelBPETokenizer(unicode_normalizer="nfkc", lowercase=True)

In [30]:
# Train it
tokenizer_1.train_from_iterator(batch_iterator(), vocab_size=vocab_size, special_tokens=special_tokens)

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 25000


#### Solution 2: Define everything yourself

In [12]:
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
tokenizer = Tokenizer(BPE())

# Then we enable lower-casing and unicode-normalization
# The Sequence normalizer allows us to combine multiple Normalizer that will be
# executed in order.
tokenizer.normalizer = Sequence([NFKC(), Lowercase()])

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
tokenizer.decoder = ByteLevelDecoder()

In [21]:
from tokenizers.trainers import BpeTrainer

# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(
    vocab_size=vocab_size, show_progress=True, initial_alphabet=ByteLevel.alphabet(), special_tokens=special_tokens
)
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 25000


Et voilà ! You trained your very first tokenizer from scratch using `tokenizers`. Of course, this 
covers only the basics, and you may want to have a look at the documentation to customize your tokenizer.

Now, let see how behave our newly trained tokenizer

In [22]:
# Let's tokenizer a simple input
encoding = tokenizer.encode("This is a simple input to be tokenized")

print("Encoded string: {}".format(encoding.tokens))

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded string: ['Ġthis', 'Ġis', 'Ġa', 'Ġsimple', 'Ġinput', 'Ġto', 'Ġbe', 'Ġto', 'ken', 'ized']
Decoded string:  this is a simple input to be tokenized


The Encoding structure exposes multiple properties which are useful when working with transformers models

- normalized_str: The input string after normalization (lower-casing, unicode, stripping, etc.)
- original_str: The input string as it was provided
- tokens: The generated tokens with their string representation
- input_ids: The generated tokens with their integer representation
- attention_mask: If your input has been padded by the tokenizer, then this would be a vector of 1 for any non padded token and 0 for padded ones.
- special_token_mask: If your input contains special tokens such as [CLS], [SEP], [MASK], [PAD], then this would be a vector with 1 in places where a special token has been added.
- type_ids: If your input was made of multiple "parts" such as (question, context), then this would be a vector with for each token the segment it belongs to.
- overflowing: If your input has been truncated into multiple subparts because of a length limit (for BERT for example the sequence length is limited to 512), this will contain all the remaining overflowing parts.

## Use your brand-new tokenizer into Transformers

In [26]:
from transformers import PreTrainedTokenizerFast

tokenizer_fast = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer_1,
    unk_token="<|endoftext|>",
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
    add_prefix_space=False,
)

and save it!

In [33]:
tokenizer_fast.save_pretrained('brand-new-tokenizer')

('brand-new-tokenizer/tokenizer_config.json',
 'brand-new-tokenizer/special_tokens_map.json',
 'brand-new-tokenizer/tokenizer.json')